# Brain Training
This is the notebook where everything regarding NF Training will come together

# 1 Preparations

## 1.1 modules and where to put files
- double check all these parameters
- change sub and sess according to the current participant and session (!)
- double check the destination directory to be sure
- don't use the Template Notebook (if you do so...), but copy/paste a notebook into a subject/session specific directory!
- do not do this work just before a measurement. Have it prepared (and preferable tested!)

In [1]:
# import (most of the) modules that we need
import time
import re
import os
from multiprocessing import Process
import numpy as np
import pylsl
from datetime import datetime
from nftools import guis
from nftools.mne import no_bad_samples, no_bad_channels, detect_channel_types
from nftools.threshold import find_mode, contiguous_regions, detect_bursts, determine_optimal_threshold
import mne
import dynarray
import pickle
from scipy import io as spio
import matplotlib.pyplot as plt
from pynfb.protocols.ssd.topomap_selector_ica import ICADialog

In [2]:
# use the qt event loop, disable warnings (they flood the screen)
%matplotlib qt  
# %gui qt

import warnings; warnings.filterwarnings('ignore') 
mne.set_config('MNE_LOGGING_LEVEL', 'WARNING')

## 1.2 Define sub, ion and run numbers
- this will also load all the available data - if they've already been recorded!

In [3]:
# change these according to which sub-sess we have:

dataset='BrainTraining' # Rockhampton and Newcastle are other options for this

sub = 1
ses = 1
run = 13

In [4]:
# define where we put our stuff - prepare for BIDS Format Style
home_dir = os.path.expanduser('~')
save_dir = os.path.join(home_dir, 'nf/rawdata/{}/bids'.format(dataset))

this_save_dir = os.path.join(save_dir, 'sub-{:02d}'.format(sub), 'ses-{:02d}'.format(ses), 'eeg')

# we should also ... MAKE this savedir! If it exists, we don't do anything.
if not os.path.exists(this_save_dir):
    os.makedirs(this_save_dir)
    print("Directory " , this_save_dir ,  " Created ")
else:    
    print("Directory " , this_save_dir ,  " already exists")   
    
    
# expected files to be read/written:
fname_raw_eo_run = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'eo', run))
fname_raw_ec_run = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'ec', run))
fname_ica_ocular_rejection = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_ica-ocular-rejection_run-{:02d}.pkl'.format(sub, ses, run))
fname_csp_alpha_rejection = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_csp-alpha-rejection_run-{:02d}.pkl'.format(sub, ses, run))

fname_calibration_envelopes = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_calibration_envelopes_run-{:02d}.pkl'.format(sub, ses, run))
fname_calibration_parameters = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_calibration_parameters_run-{:02d}.pkl'.format(sub, ses, run))

fname_raw_nftraining_run = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'raw-nftraining', run))
fname_rtanalyzed_nftraining_eeg = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'rtanalyzed-nftraining-eeg', run))
fname_rtanalyzed_nftraining_emg = os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'rtanalyzed-nftraining-emg', run))


# if files exist, load them
if os.path.exists(fname_raw_eo_run):
    print('found raw eyes open: \t\t' + os.path.basename(fname_raw_eo_run))
    raw_eo = mne.io.read_raw_fif(fname_raw_eo_run, preload=True)
if os.path.exists(fname_raw_ec_run):
    print('found raw eyes closed: \t\t' + os.path.basename(fname_raw_ec_run))
    raw_ec = mne.io.read_raw_fif(fname_raw_ec_run, preload=True)
if os.path.exists(fname_ica_ocular_rejection):
    print('found ica eyeblink rejection:\t' + os.path.basename(fname_ica_ocular_rejection))
    with open(fname_ica_ocular_rejection,'rb') as f:
        ica_rejection = pickle.load(f)
if os.path.exists(fname_csp_alpha_rejection):
    print('found csp alpha rejection: \t' + os.path.basename(fname_csp_alpha_rejection))
    with open(fname_csp_alpha_rejection, 'rb') as f:
        csp_rejection = pickle.load(f)

if os.path.exists(fname_calibration_envelopes):
    print('found calibration envelopes: \t' + os.path.basename(fname_calibration_envelopes))
    with open(fname_calibration_envelopes, 'rb') as f: 
        calibration_envelopes = pickle.load(f)
    locals().update(calibration_envelopes)
if os.path.exists(fname_calibration_parameters):
    print('found calibration parameters: \t' + os.path.basename(fname_calibration_parameters))
    with open(fname_calibration_parameters, 'rb') as f: 
        calibration_parameters = pickle.load(f)
    locals().update(calibration_parameters)
        
if os.path.exists(fname_raw_nftraining_run):
    print('found raw nftraining: \t\t' + os.path.basename(fname_raw_nftraining_run))
    raw_nftraining = mne.io.read_raw_fif(fname_raw_nftraining_run)
if os.path.exists(fname_rtanalyzed_nftraining_eeg):
    print('found rt-analyzed eeg: \t\t' + os.path.basename(fname_rtanalyzed_nftraining_eeg))
    rtanalyzed_nftraining_eeg = mne.io.read_raw_fif(fname_rtanalyzed_nftraining_eeg)
if os.path.exists(fname_rtanalyzed_nftraining_emg):
    print('found rt-analyzed emg: \t\t' + os.path.basename(fname_rtanalyzed_nftraining_emg))
    rtanalyzed_nftraining_emg = mne.io.read_raw_fif(fname_rtanalyzed_nftraining_emg)        

Directory  /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg  already exists


## 1.3 Fix/check the EEG Cap

- fix the EEG Cap
- check with openBCI GUI if the signals look OK, once they do:
- run the `python raw_eo_data --stream`, followed by `/start`
- the light on the usb stick should go <font color="red">RED</font>
- (re)-start the Cap or USB if it doesn't work, followed by commands above

## 1.4 Connect to the real-time Data Stream

In [5]:
pylsl.resolve_streams()

In [6]:
# prints out which streams are currently available
stream_ids = [ pylsl.stream_inlet(s).info().source_id() for s in pylsl.resolve_streams() ]
print(stream_ids)

['openbci_eeg_id76', 'openbci_aux_id76']


- copy/paste the eeg lab name (left of the 2 outputs) into stream_id variable:
- thake the one that says **eeg**, not aux!

In [7]:
# 'subscribe' to a data stream; grab all essential(s), fs, names, etc
# stream_id = 'openbci_eeg_id134'

# eeg_stream_id = [stream_id for stream_id in stream_ids if re.match('.*_eeg_.*', stream_id)]
eeg_stream_id = [stream_id for stream_id in stream_ids if re.match('.*eeg.*', stream_id)]
if len(eeg_stream_id)>0:
    eeg_stream_id=eeg_stream_id[0]
print(eeg_stream_id)

openbci_eeg_id76


In [8]:
# try grabbing all the information from that stream:
data_stream=pylsl.resolve_byprop("source_id", eeg_stream_id, timeout=5.0)
if data_stream:
    data_inlet=pylsl.stream_inlet(data_stream[0], max_buflen=10)
    stream_info = data_inlet.info()
    stream_Fs = stream_info.nominal_srate()
    stream_xml = stream_info.desc()
    chans_xml = stream_xml.child("channels")
    chan_xml_list = []
    ch = chans_xml.child("channel")
    while ch.name() == "channel":
        chan_xml_list.append(ch)
        ch = ch.next_sibling("channel")
    channel_names = [ch_xml.child_value("label") for ch_xml in chan_xml_list]
    data_inlet_dt = data_inlet.time_correction(timeout=5.0)
    sampling_freq = data_stream[0].nominal_srate()
    print('name = %s' % data_stream[0].name())
    print('sampling_freq = %d' % sampling_freq)
    print('channel_count = %d' % data_stream[0].channel_count())
    print('channel_format = %d' % data_stream[0].channel_format())
else:
    raise Exception('No Data Stream Found - Is your EEG Cap running?')

name = openbci_eeg
sampling_freq = 125
channel_count = 16
channel_format = 1


# 2 Data acquisition for Calibrations

## 2.1 Measure Some Data (Eye Open)
- Inform that the following measurement is an eye open measurement.
- Subjects are allowed to blink as normal
- duration about 2 minutes
- might have to re-run this in order to fix a channel from being busted
- when ready press 'stop acquisition' and close window

In [9]:
# we put the incoming data in here:
np_eo = dynarray.DynamicArray((None, len(channel_names))); times_of_samples = []

# init/open the window in which we visualize data
w=guis.AcquireData(sampling_freq, channel_names)

# before we start, pull everything from the buffer (empty it)
data_inlet.pull_chunk()
while data_inlet.samples_available(): data_inlet.pull_chunk() 
    
# then start acquiring data as long as button 'stop' not pressed:
w.RUNLOOP=True
while w.RUNLOOP:

    time.sleep(0.001)
    if not data_inlet.samples_available():
        w.update(None)
    else:
        chunk_data, chunk_times = data_inlet.pull_chunk(timeout=0.0) # grab from LSL

        
        np_eo.extend(chunk_data) # add to our list
        times_of_samples.append(chunk_times)
        w.update(chunk_data) # update the GUI window       


In [10]:
# and we make the MNE data file from it
raw_eo = mne.io.RawArray(np.transpose(np_eo)*1E-6,
                        mne.create_info(channel_names, 
                                    sampling_freq, 
                                    detect_channel_types(channel_names), 
                                    'standard_1020'
                        )
                   )

# we make an lsl packet membership so we can check transmission statistics
lsl_packet = np.array([i 
          for i, c in enumerate([len (x) for x in times_of_samples]) 
          for n in range(c)]
        )

# make a lsl timestamp signal (essential later on)
lsl_time = np.concatenate(times_of_samples) - times_of_samples[0][0]

# We add this infmrmation to the raw.
info_lslinfo = mne.create_info(('lsl_time','lsl_packet'), sampling_freq, ('misc','misc'))
raw_lslinfo = mne.io.RawArray((lsl_time, lsl_packet), info_lslinfo)
raw_eo.add_channels([raw_lslinfo])

<RawArray  |  None, n_channels x n_times : 18 x 7875 (63.0 sec), ~1.1 MB, data loaded>

## 2.2 Inspect the eo data  (run1)
- scroll through the data see if the EEG signal is what you expect
- mark bad channels (that cannot be rescued)
- mark bad segments

In [11]:
# we use a simple display filter
raw_eo_copy = raw_eo.copy()
raw_eo_copy.filter(1, 40).notch_filter((25, 50)).plot(title='eyes open');

- save this raw data to disk (filename taken care of)

In [12]:
raw_eo.set_annotations(raw_eo_copy.annotations)
raw_eo.info['bads'] = raw_eo_copy.info['bads']

this_raw_fname = 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'eo', run)
fname_raw_eo_run = os.path.join(this_save_dir, this_raw_fname)

raw_eo.save(fname_raw_eo_run, overwrite=True)

print('saved: ' + fname_raw_eo_run)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_task-eo_run-13.fif


## 2.3 Measure Some Data (Eye Closed)
- Inform your subject to keep the eye closed, then start the measurement
- duration about 1 minute

In [14]:
# we put the incoming data in here:
np_ec = dynarray.DynamicArray((None, len(channel_names))); times_of_samples = []


# init/open the window in which we visualize data
w=guis.AcquireData(sampling_freq, channel_names)

# before we start, pull everything from the buffer (empty it)
data_inlet.pull_chunk()
while data_inlet.samples_available(): data_inlet.pull_chunk() 
    
# then start acquiring data as long as button 'stop' not pressed:
w.RUNLOOP=True
while w.RUNLOOP:

    time.sleep(0.001)
    if not data_inlet.samples_available():
        w.update(None)
    else:
        chunk_data, chunk_times = data_inlet.pull_chunk(timeout=0.0) # grab from LSL

        np_ec.extend(chunk_data) # add to our list
        times_of_samples.append(chunk_times)
        w.update(chunk_data) # update the GUI window        

In [15]:
# and we make the MNE data file from it        
raw_ec = mne.io.RawArray(np.transpose(np_ec)*1E-6,
                        mne.create_info(channel_names, 
                                    sampling_freq, 
                                    detect_channel_types(channel_names), 
                                    'standard_1020'
                               )
                       )
raw_ec.info['bads'] = raw_eo.info['bads']
print('setting bad chanels in the data set to be the same as previous dataset')
print(raw_ec.info['bads'])
print('Do not change bad channels further - re-do eo and ec if needed!')

# we make an lsl packet membership so we can check transmission statistics
lsl_packet = np.array([i 
          for i, c in enumerate([len (x) for x in times_of_samples]) 
          for n in range(c)]
        )

# make a lsl timestamp signal (essential later on)
lsl_time = np.concatenate(times_of_samples) - times_of_samples[0][0]

# We add this information to the raw.
info_lslinfo = mne.create_info(('lsl_time', 'lsl_packet'), sampling_freq, ('misc','misc'))
raw_lslinfo = mne.io.RawArray((lsl_time, lsl_packet), info_lslinfo)
raw_ec.add_channels([raw_lslinfo])

setting bad chanels in the data set to be the same as previous dataset
[]
Do not change bad channels further - re-do eo and ec if needed!


<RawArray  |  None, n_channels x n_times : 18 x 7137 (57.1 sec), ~1.0 MB, data loaded>

## 2.4 Inspect the ec data
- scroll through the data see if the EEG signal is what you expect
- mark bad channels (that cannot be rescued)
- mark bad segments

In [16]:
# we use a simple display filter
raw_ec_copy = raw_ec.copy()
raw_ec_copy.filter(1, 35).plot(title='eyes closed');

In [17]:
# check bad channels in eo, compare to ec
if not raw_ec_copy.info['bads'] == raw_eo.info['bads']:
    # raise Exception('Bad Channels are not the same between the two datasets - fix this first')
    all_bads = list(set(raw_ec_copy.info['bads'] + raw_eo.info['bads']))
    raw_eo.info['bads'] = all_bads
    raw_ec.info['bads'] = all_bads
    
    # save raw_eo again
    raw_eo.save(fname_raw_eo_run, overwrite=True)
    print('saved AGAIN: ' + fname_raw_eo_run)

# - save this raw data to disk (filename taken care of)

raw_ec.set_annotations(raw_ec_copy.annotations)
raw_ec.info['bads'] = raw_ec_copy.info['bads']

this_raw_fname = 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'ec', run)
fname_raw_ec_run = os.path.join(this_save_dir, this_raw_fname)

raw_ec.save(fname_raw_ec_run, overwrite=True)
    
print('saved: ' + fname_raw_ec_run)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_task-ec_run-13.fif


# 3 Preprocessing Calibations

## 3.1 Calculate the Eyeblink Spatial Rejection
- this will automatically read in the EO data  (run1)
- and run the ICA analysis
- your job is to select the component most resembling ocular artifact

In [18]:
bad_channel_mask = [ch not in raw_eo.info['bads'] for ch in raw_eo.ch_names[:-2]]

# apply the annotations and bad channel mask to get a matrix of 'good' data:
clean_raw = no_bad_channels(
    no_bad_samples(
        raw_eo.copy().filter(3, 40, picks='eeg').notch_filter((25, 31.25, 50))
    )
)

# run the ICA analysis -- the ICA analysis already applies a 3-40 Hz filter!!
from pynfb.protocols.ssd.topomap_selector_ica import ICADialog

ica_rejection, _, _, ica_unmixing_matrix, _, _ = ICADialog.get_rejection(
    clean_raw.get_data(picks='eeg').T, 
    clean_raw.ch_names[:-2],  # do not consider the last 2 channels in this calculation
    clean_raw.info['sfreq'],
    decomposition=None
)

bad_channel_mask = [ch not in raw_eo.info['bads'] for ch in raw_eo.ch_names[:-2]]
ica_rejection = ica_rejection.expand_by_mask(bad_channel_mask)

print('Created an ICA Spatial Filter')
print(ica_rejection)

apply filter: 3 to 45
Dropped 132 outliers
ICA/CSP time elapsed = 4.806611061096191s
Table drawing time elapsed = 4.550941467285156s
Created an ICA Spatial Filter


## 3.2 Save the eyeblink Rejection
- close all open windows

In [19]:
# we save the ICA for ocular rejection:
# save the ICA rejection 
this_raw_fname = 'sub-{:02d}_ses-{:02d}_ica-ocular-rejection_run-{:02d}.pkl'.format(sub, ses, run)
fname_ica_ocular_rejection = os.path.join(this_save_dir, this_raw_fname)

with open(fname_ica_ocular_rejection, 'wb') as f: pickle.dump(ica_rejection, f)
print('saved: ' + fname_ica_ocular_rejection)

# save it also as a .txt matrix (for matlab)
np.savetxt(re.sub('.pkl$','.txt', fname_ica_ocular_rejection), ica_rejection.val)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_ica-ocular-rejection_run-13.pkl


## 3.3 Check the eyeblink Rejection

In [20]:
# plot before ICA
ica_compare = no_bad_samples(raw_eo.copy().filter(1, None).notch_filter((25, 31.25, 50)))
ica_compare.drop_channels(('lsl_time','lsl_packet'))

# plot after ICA
after_ica = mne.io.RawArray(ica_rejection.apply(ica_compare.get_data().T).T, 
        mne.create_info(['ica-{}'.format(n) for n in ica_compare.ch_names], 
                        ica_compare.info['sfreq'],
                        ['eeg' for i in range(len(ica_compare.ch_names))],
                        None)
               )
after_ica.set_annotations(raw_eo.annotations)

after_ica.filter(1, None).notch_filter((25, 31.25, 50))

ica_compare.add_channels([after_ica])
ica_compare.plot(n_channels=32);

## 3.4 Handle Alpha Power Rejection

In [21]:
# we concatenate the 'eo' and the 'ec' data --> make eoec dataset
clean_raw_eo = no_bad_channels(no_bad_samples(raw_eo.copy().filter(3, None)))
clean_raw_ec = no_bad_channels(no_bad_samples(raw_ec.copy().filter(3, None)))

eoec = mne.concatenate_raws([clean_raw_eo.copy(), clean_raw_ec])
eoec.filter(None, 40).notch_filter((25, 31.25, 50)).drop_channels(('lsl_time','lsl_packet'))
eoec.crop(tmin=1, tmax=eoec.times[-1]-1) # crop it to remove edge FX


# Apply the eye blink rejection
bad_channel_mask = [ch not in eoec.info['bads'] for ch in eoec.ch_names]
shrunk_ica_rejection = ica_rejection.shrink_by_mask(bad_channel_mask)

# make the dataset - WITH ICA applied --> make eoec_ica_applied dataset
eoec_ica_applied = mne.io.RawArray(
    shrunk_ica_rejection.apply(eoec.get_data().T).T, 
    mne.create_info(['ica-{}'.format(n) for n in eoec.ch_names],
                        eoec.info['sfreq'],
                        ['eeg' for i in range(len(eoec.ch_names))],
                        None)
    )
eoec_ica_applied.set_annotations(eoec.annotations).filter(3, 40)


# make a vector: 0 for 'eyes open'; 1 for 'eye closed'
total_samples = eoec.last_samp - eoec.first_samp + 1
samples_in_eo = clean_raw_eo.last_samp - eoec.first_samp + 1
samples_in_ec = total_samples - samples_in_eo
labels_for_csp = np.hstack((np.zeros(samples_in_eo), np.ones(samples_in_ec)))

# do the Alpha Power calculation; bring up the GUI to select
csp_rejection, filter, topography, _, bandpass, to_all = ICADialog.get_rejection(
    eoec_ica_applied.get_data().T,
    eoec.ch_names, 
    eoec_ica_applied.info['sfreq'],
    mode='csp', 
    _stimulus_split=False,
    labels=labels_for_csp, # will convert to 0-1 vector for each sample in x
    marks=None)

bad_channel_mask = [ch not in raw_eo.info['bads'] for ch in raw_eo.ch_names[:-2]]
csp_rejection = csp_rejection.expand_by_mask(bad_channel_mask)

apply filter: 3 to 45
Dropped 242 outliers
ICA/CSP time elapsed = 0.014914751052856445s
Table drawing time elapsed = 6.100223779678345s


## 3.5 Save the Alpha Power Rejection
- close all open windows

In [22]:
# we save the ICA for ocular rejection:
# save the ICA rejection 
this_raw_fname = 'sub-{:02d}_ses-{:02d}_csp-alpha-rejection_run-{:02d}.pkl'.format(sub, ses, run)
fname_csp_alpha_rejection = os.path.join(this_save_dir, this_raw_fname)

with open(fname_csp_alpha_rejection, 'wb') as f: pickle.dump(csp_rejection, f)
print('saved: ' + fname_csp_alpha_rejection)

# save it also as .txt matrix (for matlab)
np.savetxt(re.sub('.pkl$','.txt', fname_csp_alpha_rejection), csp_rejection.val)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_csp-alpha-rejection_run-13.pkl


## 3.6 Inspect Alpha Power Rejection
- Check the time traces:

In [23]:
bad_channel_mask = [ch not in raw_eo.info['bads'] for ch in eoec_ica_applied.ch_names]

shrunk_csp_rejection = csp_rejection.shrink_by_mask(bad_channel_mask)

eoec_csp_and_ica_applied = mne.io.RawArray(
    shrunk_csp_rejection.apply(eoec_ica_applied.get_data().T).T,
    mne.create_info(['scp-{}'.format(n) for n in eoec_ica_applied.ch_names], 
                        eoec_ica_applied.info['sfreq'],
                        ['eeg' for ch in eoec_ica_applied.ch_names], 
                        None,
               )
)

eoec_csp_and_ica_applied.set_annotations(eoec_ica_applied.annotations).filter(3, 40)

# make one big trace with ica an csp -channels. Future could be to plot - in trace - what I extracted..
# this should be Unity_matrix - csp_removal_matrix.
ica_csp_compare=eoec.copy()
ica_csp_compare.add_channels([eoec_ica_applied, eoec_csp_and_ica_applied])
ica_csp_compare.plot(title='After ICA and CSP', n_channels=3*len(eoec.ch_names));

 - Check the power spectra

In [24]:
# Plot/compare the power spectra:
eoec_ica_applied_copy = eoec_ica_applied.copy(); 
eoec_ica_applied_copy.info=eoec.info
eoec_csp_and_ica_applied_copy = eoec_csp_and_ica_applied.copy(); 
eoec_csp_and_ica_applied_copy.info=eoec.info
fh=plt.figure(figsize=(12, 5))
ah1=plt.subplot(131); eoec.plot_psd(ax=ah1); 
ah2=plt.subplot(132); eoec_ica_applied_copy.plot_psd(ax=ah2); ah2.set_ylim(ah1.get_ylim())
ah3=plt.subplot(133); eoec_csp_and_ica_applied_copy.plot_psd(ax=ah3); ah3.set_ylim(ah1.get_ylim())
ah1.set_title('No Spatial Filter')
ah2.set_title('ICA Filter (eyeblinks gone)')
ah3.set_title('CSP Filter (alpha gone) + ICA')
spectra_plot_fname = this_raw_fname = 'sub-{:02d}_ses-{:02d}_run-{:02d}_spectra.jpg'.format(sub, ses, run)
fh.savefig(os.path.join(this_save_dir, spectra_plot_fname))
print('saved: {}'.format(os.path.join(this_save_dir, spectra_plot_fname)));

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_run-13_spectra.jpg


# 4 Talk to Stimulus Computer

## 4.1 Create Simulus Handler, I
- connect via WiFi
    - ip address = 10.42.0.1
    - host = stim-pc
    - password = PASSWORD, OR 12345678
- ideally this should already be up and running, so you can skip over these more fast

In [25]:
from callpyff import bcinetwork, bcixml
bcinet = bcinetwork.BciNetwork('20.100.0.2', bcinetwork.FC_PORT, bcinetwork.GUI_PORT, 'bcixml')

## 4.2 Start Stimulus Marker Receiver
 - this will listen on UDP port 6500 for any incoming markers
 - This is to convert signals from the Presentation into annotations
 - grab markers in the NF loop with: `while not marker_queue.empty():`

In [26]:
import netifaces
#default_interface=netifaces.gateways()['default'][netifaces.AF_INET][1]
#ip_address=netifaces.ifaddresses(default_interface)[netifaces.AF_INET][0]['addr']
ip_address='20.100.0.1'
port = 6500
#print('Tell Stimulus computer to send markers to our interface {} with ip={} and port={}'.format(default_interface, ip_address, port))

bcinet.send_signal(bcixml.BciSignal({'EVENT_destip': ip_address},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'EVENT_destport': port},None, bcixml.INTERACTION_SIGNAL))

In [27]:
from multiprocessing import Process, Queue
import socket
import time

def my_marker_server(marker_queue, PORT):
    # simple markers server.
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind(('', PORT))
    print("Listening on {}:{}".format('', PORT))
    while True:
        data = sock.recv(1024)
        if data == b'stop_server':
            break
        marker_queue.put(data)

    print('finished')
    sock.close()
    
def send_stop_it(port):
    sock=socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.sendto(b'stop_server', ('localhost', port))
    sock.close()
    time.sleep(0.5)

if not 'marker_queue' in locals():
    marker_queue = Queue()
if 'my_udp_server' in locals():
    # we stop the marker server
    send_stop_it(port)
    try:
        # and we try to join it
        my_udp_server.join()
    except:
        del(my_udp_server)
        
my_udp_server = Process(target=my_marker_server, args=(marker_queue, port))
my_udp_server.start()

Listening on :6500


# 5 Define RT Signal analysis flow

##  5.1 RT Signal Analysis

In [28]:
from pynfb.signal_processing.filters import (FilterSequence, 
                                             CFIRBandEnvelopeDetector, 
                                             ExponentialSmoother,
                                             SpatialFilter,
                                             ButterFilter,
                                             ButterBandEnvelopeDetector,
                                             ScalarButterFilter,
                                             MASmoother,
                                             FFTBandEnvelopeDetector,
                                             NotchFilter
                                            )

In [29]:
# Define Filter Sequence for NF

# Which channel do we select for the EEG - we do C3.
rt_eeg_channels = ['C4']
rt_eeg_channels_mask = np.where([ch in rt_eeg_channels for ch in channel_names], 1, 0)/len(rt_eeg_channels)
eeg_channel_select = SpatialFilter(rt_eeg_channels_mask)
highpass_filter = ScalarButterFilter([3, None], sampling_freq)
butter_filter = ScalarButterFilter([12, 15], sampling_freq)

eeg_notch_50 = NotchFilter(50, sampling_freq, len(channel_names))
# eeg_notch_25 = NotchFilter(25, sampling_freq, len(channel_names))

preprocess_filters_eeg = FilterSequence([
    ica_rejection,
    csp_rejection,
    eeg_channel_select,
    highpass_filter
    
])
envelope_filter_eeg = CFIRBandEnvelopeDetector([12, 15], sampling_freq, MASmoother(round(sampling_freq/5)))
butter_visualization_eeg = ButterFilter([12, 15], sampling_freq, 1)


# Processing of the EMG - this is basically our second channel...
rt_emg_channels = ['T3','T4','Fp1','Fp2']
# rt_emg_channels = ['T7','T8','TP9','TP10']
rt_emg_channels_mask = np.where([ch in rt_emg_channels for ch in channel_names], 1, 0)/len(rt_emg_channels)
emg_channel_select = SpatialFilter(rt_emg_channels_mask)

preprocess_filters_emg = FilterSequence([
    eeg_notch_50,
    emg_channel_select,
])

envelope_filter_emg = ButterBandEnvelopeDetector([30, 62], sampling_freq, MASmoother(round(sampling_freq/5)))
butter_visualization_emg = ButterFilter([30, 60], sampling_freq, 1)

## 5.2 Appy Rt Filters on Calibration Data
This will:
- apply the RT filters to our existing data
- calibrate the signaltracking objects (that send stuff to the Stimulus) properly
- set all the scalings

In [32]:
# eoec_rawraw = mne.concatenate_raws([raw_eo.copy(), raw_ec])
# in order to determine threshold for EEG and EMG -- use ONLY (!) the raw_eo!


# eoec_rawraw = raw_eo.copy().drop_channels(('lsl_time','lsl_packet'))
eoec_rawraw = mne.concatenate_raws([raw_eo.copy(), raw_ec]).drop_channels(('lsl_time','lsl_packet'))

# eoec_rawraw.plot();

preprocessed_eeg = preprocess_filters_eeg.apply(eoec_rawraw.get_data().T)*1E6
preprocessed_emg = preprocess_filters_emg.apply(eoec_rawraw.get_data().T)*1E6

eeg_vec = envelope_filter_eeg.apply(preprocessed_eeg)
emg_vec = envelope_filter_emg.apply(preprocessed_emg)


In [33]:
# save the eeg_vec and emg_vec also as .pkl and .mat file
calibration_envelopes = {'eeg_vec':eeg_vec, 'emg_vec':emg_vec, 'preprocessed_eeg': preprocessed_eeg}

# save it
this_raw_fname = 'sub-{:02d}_ses-{:02d}_calibration_envelopes_run-{:02d}.pkl'.format(sub, ses, run)
fname_calibration_envelopes = os.path.join(this_save_dir, this_raw_fname)
with open(fname_calibration_envelopes, 'wb') as f: pickle.dump(calibration_envelopes, f)
print('saved: ' + fname_calibration_envelopes)
spio.savemat(re.sub('.pkl$','.mat', fname_calibration_envelopes), calibration_envelopes)
print('saved: ' + re.sub('.pkl$','.mat', fname_calibration_envelopes))

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_calibration_envelopes_run-13.pkl
saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_calibration_envelopes_run-13.mat


## 5.3 Use calibration data to set thresholds

In [34]:
butter_filter = ScalarButterFilter([12, 15], sampling_freq)

mode_eeg = find_mode(butter_filter.apply(preprocessed_eeg), make_plot=True)
median_eeg = np.median(eeg_vec)
mean_eeg = np.mean(eeg_vec)
print('eeg: mean {:2.3f}, median {:2.3f}, mode {:2.3f}: '.format(mean_eeg, median_eeg, mode_eeg))
mode_fig = plt.gcf()

mode_emg = None # we won't bother detecting mode of EMG with poor quality EMG
median_emg = np.median(emg_vec)
mean_emg = np.mean(emg_vec)
print('emg: mean {:2.3f}, median {:2.3f}, mode {:2.3f}: '.format(mean_emg, median_emg, 0))

print('{:3.2f} minutes of data'.format(len(preprocessed_eeg)/sampling_freq/60))

# determine the threshold for 90 bursts!
mode_thr_eeg, _, _ = determine_optimal_threshold(butter_filter.apply(preprocessed_eeg), sampling_freq, 0.2, 90, make_plot=True)
print('mode_thr_eeg = {:2.3f}'.format(mode_thr_eeg))
mode_thr_fig = plt.gcf()

median_thr_emg = 10.

EEG_THR = mode_thr_eeg * mode_eeg
EEG_DUR = 0.20
EEG_STSCALING = 10 * mode_eeg

EMG_THR = median_thr_emg * median_emg
EMG_DUR = 0.25
EMG_STSCALING = 2 * EMG_THR



calibration_parameters = {'mode_eeg':mode_eeg,
                          'mode_emg':0.,
                          'mode_thr_eeg':mode_thr_eeg,
                          'median_thr_emg':median_thr_emg,
                          'EEG_THR':EEG_THR, 
                          'EEG_DUR':EEG_DUR, 
                          'EEG_STSCALING': EEG_STSCALING, 
                          'EMG_THR':EMG_THR,
                          'EMG_DUR':EMG_DUR,
                          'EMG_STSCALING':EMG_STSCALING
                         }

# save those too
this_raw_fname = 'sub-{:02d}_ses-{:02d}_calibration_parameters_run-{:02d}.pkl'.format(sub, ses, run)
fname_calibration_parameters = os.path.join(this_save_dir, this_raw_fname)
with open(fname_calibration_parameters, 'wb') as f: pickle.dump(calibration_parameters, f)
print('saved: ' + fname_calibration_parameters)
spio.savemat(re.sub('.pkl$','.mat', fname_calibration_parameters), calibration_parameters)
print('saved: ' + re.sub('.pkl$','.mat', fname_calibration_parameters))

mode_fig.savefig(os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_mode_run-{:02d}.png'.format(sub, ses, run)))
mode_thr_fig.savefig(os.path.join(this_save_dir, 'sub-{:02d}_ses-{:02d}_threshold_run-{:02d}.png'.format(sub, ses, run)))

eeg: mean 8.654, median 4.176, mode 2.973: 
emg: mean 3.841, median 3.061, mode 0.000: 
2.00 minutes of data
mode_thr_eeg = 2.042
saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_calibration_parameters_run-13.pkl
saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_calibration_parameters_run-13.mat


## 5.4 Create Stimulus Handlers, II

In [35]:
# these objects can send variables over to the stimulus
# parameters to convert the filtered EEG signal to the stimulus
# the following parameter should come out of the EEG data, as our initial threshold to use:
# global_std_band=5

from nftools.nftools import signaltracking
track_for_eeg_stimuli = signaltracking.sending_to_nfstim(
    sampling_freq,
    thr=EEG_THR, 
    dur=EEG_DUR, 
    feedback_type='eeg', 
    max4audio=1.2, 
    bcinet=bcinet, 
    st_scaling=10,
    verbose=False
)

# parameters to convert the filtered EMG signal to the stimulus
track_for_emg_stimuli = signaltracking.sending_to_nfstim(
    sampling_freq,
    thr=EMG_THR, 
    dur=EMG_DUR, 
    feedback_type='emg', 
    bcinet=bcinet, 
    st_scaling=EMG_STSCALING,
    verbose=False
)

thr: 6.07, dur: 0.20
bcinet is passed on
thr: 30.61, dur: 0.25
bcinet is passed on


# 6 The Training

## 6.1 Start the NF Stimulation
 - you still have to press <ENTER> to actually start the stimulus, but Not Yet!!

In [36]:
print(bcinet.getAvailableFeedbacks())

['TestD2', 'MovingRhomb', 'EOEC', 'LibetClock', 'BrainWaveTraining_II', 'TobiQLAdapter', 'EyetrackerRawdata', 'EyetrackerFeedback', 'HexoSpeller', 'P300_Rectangle', 'ERPHex', 'BrainWaveTraining', 'StopVigilanceTask', 'FeedbackCursorArrow', 'TrivialPong', 'CheckerboardVEP', 'HexoSpellerVE', 'BoringClock', 'nback_verbal', 'VisualOddball', 'BrainPong', 'CakeSpellerVE', 'MovingRhombGL', 'RestingState', 'NFBasicThermometer', 'RSVPSpeller', 'EEGfMRILocalizer', 'Oddball', 'Lesson01b', 'GoalKeeper', 'CenterSpellerVE', 'MultiVisualOddball', 'StroopFeedback', 'ERPMatrix', 'Lesson04', 'Lesson05', 'Lesson06', 'Lesson01', 'Lesson02', 'Lesson03', 'VisualOddballVE']


In [37]:
bcinet.send_init('BrainWaveTraining_II')

In [38]:
# tell the stimulus about the monitor
bcinet.send_signal(bcixml.BciSignal({'EX_TESTNFNOISE': False},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'EX_PR_SLEEPTIME': 0.005},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_PIXWIDTH': 1366},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_PIXHEIGHT': 768},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_FULLSCR': True},None, bcixml.INTERACTION_SIGNAL))

In [39]:
bcinet.play()

## 6.2 make the windows for plotting
- move them somewhere convenient

In [40]:
# create our UI 'Experience' -- it can consist of 3 separate, movable windows (for now)
# same window as before + 2 other windows - 1 for interaction with stim/thresholds; 1 for looking
# at the analysis itself.

w_acquire = guis.AcquireData(sampling_freq, channel_names)
w_interaction = guis.NFChangeThresholds(track_for_eeg_stimuli, track_for_emg_stimuli)
w_eeganalysis = guis.AnalyzeData(sampling_freq, ['EEG','env','thr','vmarker','amarker'],track_for_eeg_stimuli)
w_emganalysis = guis.AnalyzeData(sampling_freq, ['EMG','env','thr','vmarker','amarker'],track_for_emg_stimuli)

from PyQt5 import QtWidgets, QtGui
class AllInOneWindow(QtWidgets.QWidget):
    def __init__(self):
        super(AllInOneWindow, self).__init__()
        layout = QtWidgets.QGridLayout()
        layout.addWidget(w_acquire, 1, 1, 1, 1)
        layout.addWidget(w_interaction, 2, 1, 1, 1)
        layout.addWidget(w_eeganalysis, 1, 2, 1, 1)
        layout.addWidget(w_emganalysis, 2, 2, 1, 1)
        
        # remove the checkboxes of all except the EEG analysis window:
        w_acquire.datawidget.plot_check_box.setChecked(False)
        w_emganalysis.plot_check_box.setChecked(False)
        
        self.setLayout(layout)
        self.show()


# can we combine everything into 1 Big Gui -- YES
big_window = AllInOneWindow()

## 6.3 Staring the NF Loop

In [41]:
# clear EEG Data buffer
data_inlet.pull_chunk()
while data_inlet.samples_available(): data_inlet.pull_chunk() 

# containers for data collection: 

data_nf = dynarray.DynamicArray((None, len(channel_names))); times_of_samples = []
data_analysis_eeg = dynarray.DynamicArray((None, 5))
data_analysis_emg = dynarray.DynamicArray((None, 5))

# collect markers from the Stimulation:
stim_Annotations = mne.Annotations(0, 0, 'Start NF Loop')

# collect markers from the interaction GUI:
w_interaction.GUI_Annotations.append(time.time()-w_interaction.begin_time, 0, 'startloop')

# start the loop
w_acquire.RUNLOOP=True
acquisition_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
t_begin=time.time()
while w_acquire.RUNLOOP:
    
    # wait 1 msec
    time.sleep(0.001)
    
    
    # check if the presentation gave any markers, and collect them
    while not marker_queue.empty():
        stim_Annotations.append(time.time()-t_begin, 0, marker_queue.get().decode())
    
    # handle the incoming data (if any):
    if not data_inlet.samples_available(): 
        w_acquire.update(None)
    else:
        chunk_data, chunk_times = data_inlet.pull_chunk() # grab from LSL
        
        # how to deal with missing data -- do we need to insert any placeholder values?
        
        

        # update signal window so we can see raw signals
        w_acquire.update(chunk_data) 
        
        # store the raw data (and times)
        times_of_samples.append(chunk_times)
        data_nf.extend(chunk_data)

        
        #
        # EEG Signal processing
        #
        
        # apply spatial and temporal filters to the raw signal; for EEG and EMG:
        preprocessed_eeg = preprocess_filters_eeg.apply(chunk_data)        
        envelope_eeg = envelope_filter_eeg.apply(preprocessed_eeg)


        # check if it's above threshold or not; send markers to stimulus computer
        visual_markers_eeg, audio_markers_eeg = track_for_eeg_stimuli.check_above_threshold(envelope_eeg)  
        # send the signal  to stimulus computer, too
        track_for_eeg_stimuli.send_data_signal(envelope_eeg) 
        

        # visualize the processing steps
        # analysis_names_eeg = ('EEG', 'env', 'thr', 'vmarker', 'amarker')
        analysis_data_eeg = np.vstack((
            np.abs(butter_visualization_eeg.apply(preprocessed_eeg[:, None])[:, 0]),
            envelope_eeg,
            track_for_eeg_stimuli.thr * np.ones(preprocessed_eeg.shape),
            visual_markers_eeg,
            audio_markers_eeg,
        )).T
        
        data_analysis_eeg.extend(analysis_data_eeg)  # store it for later conversion
        w_eeganalysis.update(analysis_data_eeg) # update analysis window

        
        #
        # EMG Signal Processing
        #
        
        # do the same for the EMG NF, too:
        # apply spatial and temporal filters to the raw signal; for EEG and EMG:
        preprocessed_emg = preprocess_filters_emg.apply(chunk_data)
        envelope_emg = envelope_filter_emg.apply(preprocessed_emg)
        
        # check if it's above threshold or not; send markers to stimulus computer
        visual_markers_emg, audio_markers_emg = track_for_emg_stimuli.check_above_threshold(envelope_emg) 
        # send the signal  to stimulus computer, too
        track_for_emg_stimuli.send_data_signal(envelope_emg) 

        # visualize the processing steps
        # analysis_names_emg = ('EMG', 'env', 'thr', 'vmarker', 'amarker')
        analysis_data_emg = np.vstack((
            np.abs(butter_visualization_emg.apply(preprocessed_emg[:, None])[:, 0]),
            envelope_emg,
            track_for_emg_stimuli.thr * np.ones(preprocessed_emg.shape),
            visual_markers_emg,
            audio_markers_emg,
        )).T
        
        data_analysis_emg.extend(analysis_data_emg)  # store it for later conversion
        w_emganalysis.update(analysis_data_emg) # update analysis window


it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is smaller than needed: -1.666666666666667
it should never happen that a marker is sent when the length is 

## 6.4 Now the participant will do the training
- start it up on the NF Laptop now
- wait until the training is done
- then stop the loop as normal

## 6.5 make the data structure

In [42]:
# and we make the MNE data file from it        
raw_nftraining = mne.io.RawArray(np.transpose(data_nf)*1E-6,
                        mne.create_info(channel_names, 
                                    sampling_freq, 
                                    detect_channel_types(channel_names), 
                                    'standard_1020'
                           )
                     )

# attribute a number (1, 2, 3, ... etcetc) to each incoming packet of samples
lsl_packet = np.array([i 
          for i, c in enumerate([len (x) for x in times_of_samples]) 
          for n in range(c)]
        )

# make a lsl timestamp signal (essential later on)
lsl_time = np.concatenate(times_of_samples) - times_of_samples[0][0]

# We add this information to the raw.
info_lslinfo = mne.create_info(('lsl_time', 'lsl_packet'), sampling_freq, ('misc','misc'))
raw_lslinfo = mne.io.RawArray((lsl_time, lsl_packet), info_lslinfo)
raw_nftraining.add_channels([raw_lslinfo])

<RawArray  |  None, n_channels x n_times : 18 x 62778 (502.2 sec), ~8.7 MB, data loaded>

## 6.6 annotation(s) of the EEG
The following sections are all intended to annotate the EEG with:
- when does the trial start and end?
- when did the presentation give an audio signal?
- when did the presentation give a video signal?
- when did you (as the experimenter) press any button on the GUI? What value?
- when did the LSL datastream break up (bad segment)?

You can just shift-Enter through them all

All of these items will be detected from various annotation variables and
data accumulated during the training run. All raw data will also be saved.
They will all be converted to annotations that will be put into the EEG trace.
One figure will be created to illustrate (and double check) the datastream.

In [43]:
# handle annotations, since w_interaction starts before data acquisition
annots = w_interaction.GUI_Annotations.copy()
tdelta = datetime.strptime(acquisition_start,'%Y-%m-%d %H:%M:%S.%f').timestamp() - annots.orig_time
annots.onset -= tdelta
annots.orig_time=None

# grab from those the staring ones - change their times...
starting_annots = annots[list(map(lambda d: re.match('start.*', d) is not None, annots.description))]
starting_annots.onset = 0.1*np.ones(len(starting_annots))
starting_annots.orig_time = None

In [45]:
# to extract
to_process = {'irest':'11', 'brest':'12','erest':'13', 
 'itrain':'31', 'btrain':'32','etrain':'33', 
 'itransfer':'41', 'btransfer':'42','etransfer':'43',
 'hit':'170', 'p':'171', 'emg':'230'}

# bookkeeping on the annotations! - make things nice, near, easy to understand...
new_annots = mne.Annotations(0, 0, ''); new_annots.delete(0)

for key, value in to_process.items():
    tmp_annots = stim_Annotations[[i for i, d in enumerate(stim_Annotations.description) if re.fullmatch(d, value)]]
    
    for item in tmp_annots:
        # this will deal with train, transfer and rest blocks:
        if item['description'] in ['11', '31', '41']:
            # its an instruction
            new_annots.append(item['onset'], 0, key)
        if item['description'] in ['12', '32', '42']:
            # it's the onset of a rest/training/transfer interval - where is the end?
            to_find = '{}3'.format(item['description'][0])
            # find all endings of current trans/train/rest:
            all_endings = np.array([item['onset'] for i, item in enumerate(stim_Annotations) if item['description'] == to_find])
            # find the one ending that we need:
            this_duration = all_endings[next((i for i, j in enumerate(all_endings > item['onset']) if j), [])] - item['onset']
            # if we find it, complete block: add annotation.
            if this_duration:
                new_annots.append(item['onset'], this_duration, key[1:])

        # this deals with NF hits and misses:
        if item['description'] in ['170','171']:
            new_annots.append(item['onset'], 0, key)
            
        # this deal with the EMG:
        if item['description'] in ['230']:
            new_annots.append(item['onset']-0.5, 1.0, 'BAD_' + key)
            
# bad segment annotations:
new_annots = mne.Annotations(0, 0, ''); new_annots.delete(0)
            
# these are all annotations so far - we're going to correct their onsets:            
all_annotations = starting_annots + annots + new_annots

# put the markers at the right spot by median filter of delay w.r.t. naive time:
uncorrected_time = raw_nftraining.times
from scipy.ndimage import median_filter

delay_model=median_filter(lsl_time - uncorrected_time, round(1.0*sampling_freq))
corrected_time = uncorrected_time + delay_model

# this seems a bad way to loop over the annotations - but it is
# the only way to actually change the onset timings (normal looping - doesn't work!)
for i in range(len(all_annotations)):
    uncorrected_timestamp = all_annotations.onset[i]
    corrected_index = np.argmin(abs(corrected_time - uncorrected_timestamp))
    corrected_timestamp = corrected_index / sampling_freq
    all_annotations.onset[i] = corrected_timestamp

In [47]:
plt.figure()
plt.plot(delay_model)

In [48]:
# determine segments that are bad, and annotate them too:
wsize = round(0.5*sampling_freq)
# to facilitate sliding window selection with indices
sliding_inds = iter([range(s, s+wsize) for s in range(len(delay_model)-wsize)])
# determine slope and intercept in sliding window
slopes, intercepts = np.array([np.polyfit(inds, delay_model[inds], 1) for inds in sliding_inds]).T
# the threshold (works well)
thr = np.median(abs(slopes)) * 6.0 
# true/false array of bad points -- but broaden it a little bit more (1 second)
bad_points=np.convolve(slopes > thr, np.ones(round(sampling_freq), dtype='bool'), mode='same')

In [56]:
# make our dictionary -> later will be converted towards annotation(s)
bad_segments = []
# a list of index items
cur_prev_inds = iter([(s, s+1) for s in range(len(bad_points)-1)])
for i, (cur, nxt) in enumerate(cur_prev_inds):
    
    if bad_points[nxt] and not bad_points[cur] or bad_points[cur] and i == 0:
        # we need to add half length of window size for getting the real start
        bad_segments.append({'start':i + round(wsize/2), 'duration':0})
    if bad_points[nxt] and bad_points[cur]:
        bad_segments[-1]['duration'] += 1

In [57]:
        
# insert these bad segments as annotations
# no tdelay correction necessary -- we know the samples
bad_segment_annots = mne.Annotations(0, 0, ''); bad_segment_annots.delete(0)
for segment in bad_segments:
    bad_segment_annots.append(segment['start']/sampling_freq, segment['duration']/sampling_freq, 'BAD_Transmission')

all_annotations += bad_segment_annots

In [58]:
# this takes few seconds
fig=plt.figure(figsize=(15, 5))
plt.plot(lsl_time-uncorrected_time)
ylims = plt.ylim()
plt.xlim(0, len(lsl_time))
plt.xlabel('sample'); plt.ylabel('delay (s)')

for segment in bad_segments:
    plt.vlines(segment['start']+segment['duration']/2, *ylims)
fname = 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}_datapackets.jpg'.format(sub, ses, 'raw-nftraining', run)
plt.title(fname)
fig.tight_layout()

plt.savefig(os.path.join(this_save_dir, fname))
# plt.close(fig)

## 6.7 Saving the data structures

In [59]:
# saving the raw data - as of yet, NO annotations!
this_raw_fname = 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'raw-nftraining', run)
fname_raw_nftraining_run = os.path.join(this_save_dir, this_raw_fname)
raw_nftraining.set_annotations(all_annotations)
raw_nftraining.save(fname_raw_nftraining_run, overwrite=True)
print('saved: ' + fname_raw_nftraining_run)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_task-raw-nftraining_run-13.fif


In [60]:
# and we make the MNE data file from it
analysis_names_eeg = ('EEG', 'env', 'thr', 'vmarker', 'amarker')
rtanalyzed_nftraining_eeg = mne.io.RawArray(np.transpose(np.array(data_analysis_eeg) * [1E-6, 1E-6, 1E-6, 1, 1]),
                        mne.create_info(analysis_names_eeg, 
                                    sampling_freq, 
                                    ['eeg','eeg','eeg','stim','stim'], 
                                    None)
                       )
this_raw_fname = 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'rtanalyzed-nftraining-eeg', run)
fname_rtanalyzed_nftraining_eeg = os.path.join(this_save_dir, this_raw_fname)

rtanalyzed_nftraining_eeg.set_annotations(all_annotations)
rtanalyzed_nftraining_eeg.save(fname_rtanalyzed_nftraining_eeg, overwrite=True)
print('saved: ' + fname_rtanalyzed_nftraining_eeg)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_task-rtanalyzed-nftraining-eeg_run-13.fif


In [61]:
# and we make the MNE data file from it
analysis_names_emg = ('EMG', 'env', 'thr', 'vmarker', 'amarker')
rtanalyzed_nftraining_emg = mne.io.RawArray(np.transpose(np.array(data_analysis_emg) * [1E-6, 1E-6, 1E-6, 1, 1]),
                        mne.create_info(analysis_names_emg, 
                                    sampling_freq, 
                                    ['emg','emg','emg','stim','stim'], 
                                    None)
                       )
this_raw_fname = 'sub-{:02d}_ses-{:02d}_task-{}_run-{:02d}.fif'.format(sub, ses, 'rtanalyzed-nftraining-emg', run)
fname_rtanalyzed_nftraining_emg = os.path.join(this_save_dir, this_raw_fname)


rtanalyzed_nftraining_emg.set_annotations(all_annotations)
rtanalyzed_nftraining_emg.save(fname_rtanalyzed_nftraining_emg, overwrite=True)
print('saved: ' + fname_rtanalyzed_nftraining_emg)

saved: /home/johan/nf/rawdata/BrainTraining/bids/sub-01/ses-01/eeg/sub-01_ses-01_task-rtanalyzed-nftraining-emg_run-13.fif


## Done!
The following is just to plot some of the EEG traces

In [62]:
raw_nftraining.copy().filter(3, 40).notch_filter((25, 50)).plot();

In [161]:
rtanalyzed_nftraining_emg.plot();